In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
import pickle
import math 

In [2]:
def drop_one_airport(adj,name):
    index_row = adj.columns.get_loc(name)
    adj[name]=0
    adj.loc[index_row]=0
    return adj.to_numpy()
    
adj = pd.read_csv('./init_flight_capacity.csv')
# AAL is the airport to cutted all connection
adj_new = drop_one_airport(adj,'HKG')
# adj_new.shape

In [3]:
# adj_matrix contain the flight info with passenger num
# when cutted, rmb to change this file
adj_matrix = np.load("init_flight_capacity.npy")
# # adj_matrix = adj_new

with open("terminal_name.txt") as f:
    airport_id = f.readlines()
airport_id = [i.replace("\n", "") for i in airport_id]

with open('city_id2airports.pickle', 'rb') as handle:
    cityid2airportid = pickle.load(handle)

with open('cityid2population.pickle', 'rb') as handle:
    cityid2population = pickle.load(handle)
    
with open('cityid_passengers_from_to.pickle', 'rb') as handle:
    cityid_outprob = pickle.load(handle)
flights_data = pd.read_csv("flights.csv")
# flights_data
# adj_matrix.shape

In [4]:
# fix nan
# cityid2population_e[3442] = 8604203.0
# cityid2population_e[86] = 7398000.0
# cityid2population_e[29867] = 593754.0


In [5]:
# cityid2airportid = {2451:["AAR", "AAL"],719:["ABI"] }       
# cityid2population = {2451: 237551.0, 719: 693494.0}
# cityid_outprob = {2451: {"AAR_PER": 0.4, "AAR_ABI": 0.3, "AAL_ABI":0.3}, 719: {"ABI_AAR": 1.0}}

beta_city = [0.779 for i in range(len(cityid2airportid))]
gamma_city = [0.03 for i in range(len(cityid2airportid))]
beta_flight = np.load("beta.npy") # infection rate
gamma_flight = np.copy(beta_flight)
gamma_flight[gamma_flight != 0] = 0.03 # recovery rate

In [9]:
def ppl_same_city_in_plane(airports, probabilities, infection_num_out):
    fetch = random.choices(airports, weights=probabilities, k=int(infection_num_out))

    infection_num_out_list=[]
    for item in airports:
        infection_num_out_list.append(fetch.count(item))
        
    return infection_num_out_list

def update_infection(arrive_infection_matrix):
    update_dictionary = dict(zip(airport_id, np.sum(arrive_infection_matrix, axis=0)))
    return update_dictionary

def get_infection_input(infection_dictionary):   
    city_infection_input = dict.fromkeys(cityid2airportid.keys(), 0)
    for k, v in cityid2airportid.items():
        infection_num = []
        for i in v:
            infection_num.append(infection_dictionary[i])
        city_infection_input[k] = sum(infection_num)
    return city_infection_input

def city_infection(city_infection_input, city_recovery):
    city_infection_num = np.array(list(city_infection_input.values()))
    city_population = np.array(list(cityid2population.values()))
    y_c = [(city_population - city_infection_num)/city_population, city_infection_num/city_population, city_recovery]
    
    s, i ,r = SIR(y_c, beta_city, gamma_city)
    
    after_infection = np.ceil(i*city_population)
    return r, after_infection


def flight_infection(flight_adj, flight_infection_num):

    y_f = [np.nan_to_num((flight_adj-flight_infection_num)/flight_adj,nan = 1),\
           np.nan_to_num(flight_infection_num/flight_adj, nan = 0),\
           np.zeros(len(flight_adj))]
    
    
    s, i ,r = SIR(y_f, beta_flight, gamma_flight)
    
    arrive_infection = np.nan_to_num(np.ceil(i*flight_adj), nan = 0)
    arrive_infection[arrive_infection <0 ] = 0
    infection_ppl_flight = update_infection(arrive_infection)
    
    return infection_ppl_flight

def get_on_plane(flight_infection_matrix, infection_ppl_in_airport):
    for (k, v), num_of_people in zip(cityid_outprob.items(), infection_ppl_in_airport):
        people_on_plane = ppl_same_city_in_plane(list(v.keys()), list(v.values()), num_of_people)
        flight_infection_matrix = update_flight_infection(flight_infection_matrix, list(v.keys()), people_on_plane)
    return flight_infection_matrix
    
def fly_infection_ppl(city_infection_ppl):
    return np.nan_to_num(np.ceil(city_infection_ppl/1),nan = 1)
    

def SIR(y, beta, gamma):
    S, I, R = y 
    d0 = S - beta*S*I # derivative of S(t)
    d1 = I + beta*S*I - gamma*I # derivative of I(t)
    d2 = R + gamma*I # derivative of R(t)
    
    return [d0, d1, d2]

def update_flight_infection(flight_infection, flight_com, ppl_num_list):
    flight_com = [i.split("_") for i in flight_com]
    for (flight_from, flight_to), infection_num in zip(flight_com, ppl_num_list):
        flight_from_id = airport_id.index(flight_from)
        flight_to_id = airport_id.index(flight_to)
        flight_infection[flight_from_id][flight_to_id] += infection_num
    return flight_infection
    

def loop_infection(n_iter, flight_adj):
    flight_infection_num = np.zeros((len(flight_adj), len(flight_adj)))
    #   flight_infection_num[fromcity][Tocity] = num_of_start_infection
    #   fromcity means the order from terminal_name file, AAL -> 0
    flight_infection_num[0][1] = 1
    
    city_recovery = [0.03 for i in range(len(cityid2airportid))]
    infection_ppl_list = [1]
    infection_ppl_city_list = []
    infection_ppl_flight_list = []
    
    for _ in tqdm(range(n_iter)):
        infection_ppl_flight = flight_infection(flight_adj, flight_infection_num)
        infection_ppl_flight_list.append(infection_ppl_flight)
        
        infection_ppl_city = get_infection_input(infection_ppl_flight)
        infection_ppl_city_list.append(infection_ppl_city)
        
        city_recovery, infection_ppl = city_infection(infection_ppl_city, city_recovery)
        
        flight_infection_num = get_on_plane(np.zeros((len(flight_adj), len(flight_adj))), fly_infection_ppl(infection_ppl))
        infection_ppl_list.append(np.sum(infection_ppl))
    
    return flight_infection_num, infection_ppl_list, infection_ppl_city_list, infection_ppl_flight_list
        

#  10-> loop iter, adj_matrix could be init or cutted matrix load from beginning
matrix, infection_list, infection_ppl_city_list_, infection_ppl_flight_list_ = loop_infection(10, adj_matrix)
print(infection_list) 
# print(infection_ppl_flight_list_)
matrix

  0%|          | 0/10 [00:00<?, ?it/s]

/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


[1, 4.0, 13.0, 44.0, 147.0, 493.0, 1730.0, 5894.0, 19523.0, 61806.0, 186211.0]


array([[ 0.,  7.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., 21.],
       [ 0.,  0.,  0., ...,  0., 20.,  0.]])